In [1]:
import numpy as np
import pandas as pd

data_file = '/root/autodl-tmp/Multimedia-Experiment/实验二/ColorHistogram.asc'

df = pd.read_csv(data_file, header=None, index_col=0, sep=' ')
matrix = np.array(df.iloc[:].values)
matrix.shape  # (68040, 32)  行向量

(68040, 32)

In [2]:
def compare(a, b):
    assert a.shape == b.shape
    abs_dif = np.abs(a - b)
    mean_dif = np.mean(abs_dif)
    max_dif = np.max(abs_dif)
    min_dif = np.min(abs_dif)
    print("mean_dif:{}".format(mean_dif))
    print("max_dif:{}".format(max_dif))
    print("min_dif:{}".format(min_dif))


In [3]:
def var(data, bias=False):
    # 求一组随机变量方差, data: (n * feature_dim)
    vars = []
    mean = np.mean(data, axis=0)  # (feature_dim, )
    for dim in range(data.shape[1]):
        var_dim = np.sum((data.T[dim] - mean[dim]).dot(data.T[dim] - mean[dim]))
        if bias:
            vars.append(var_dim / data.shape[0])
        else:
            vars.append(var_dim / (data.shape[0] - 1))
    return np.array(vars)


def cov(data, bias=False):
    cov = []
    mean = np.mean(data, axis=0)  # (feature_dim, )
    for i in range(data.shape[1]):
        for j in range(data.shape[1]):
            var_ij = np.sum((data.T[i] - mean[i]).dot(data.T[j] - mean[j]))
            if bias:
                cov.append(var_ij / data.shape[0])
            else:
                cov.append(var_ij / (data.shape[0] - 1))
    return np.array(cov).reshape(data.shape[1], data.shape[1])


In [4]:
from numpy import var as np_var_fuc

vars = var(matrix, bias=True)
np_var = np_var_fuc(matrix, axis=0)
compare(vars, np_var)

mean_dif:1.1570470059493743e-18
max_dif:8.673617379884035e-18
min_dif:0.0


In [5]:
from numpy import cov as np_cov_func

covs = cov(matrix, bias=False)
np_covs = np_cov_func(matrix.T, bias=False)
compare(covs, np_covs)

mean_dif:2.590361324649555e-19
max_dif:1.3877787807814457e-17
min_dif:0.0


&emsp;&emsp;给定列向量 $\boldsymbol{z}_i\in \mathcal{X} \in \mathbb{R}^{m\times 1},i=1,2,\cdots,n$，我们希望将其降至 $k$ 维。PCA 给出的方法是通过某种 *线性投影*，将高维的数据映射到低维的空间中，并期望在所投影的维度上数据的信息量最大（方差最大），以此使用较少的数据维度，同时保留住较多的原数据点的特性。  
&emsp;&emsp;样本集 $D=\{\boldsymbol{z}_i\}_{i=1}^n$ 的均值向量 $\overline{\boldsymbol{z}}=\sum\limits_{i=1}^n \boldsymbol{z}_i$，我们先将数据去中心化：$\boldsymbol{x}_i=\boldsymbol{z}_i-\overline{\boldsymbol{z}}$。其协方差矩阵为：  
$$
\boldsymbol{S}=[s_{ij}]_{m\times m}\\
s_{ij}=\frac{1}{n-1}\sum\limits_{k=1}^n (\boldsymbol{x}_{ki}-\overline{\boldsymbol{z}}_i)(\boldsymbol{x}_{kj}-\overline{\boldsymbol{z}}_j)=\frac{1}{n-1}\sum\limits_{k=1}^n \boldsymbol{x}_{ki}\cdot \boldsymbol{x}_{kj}
$$
&emsp;&emsp;所以，$\boldsymbol{S}=\frac{1}{n-1}\boldsymbol{X}\boldsymbol{X}^T$。  
&emsp;&emsp;为了降维，考虑正交线性变换：$\mathcal{W}\in \mathbb{R}^{m\times k}$，变换后的样本：$\boldsymbol{y}_i=\mathcal{W}^T\boldsymbol{x}_i \in \mathbb{R}^{k\times 1}$，变换矩阵 $\mathcal{W}$ 可认为由 $k$ 个正交基向量组成：$\mathcal{W}=(\boldsymbol{w}_1,\cdots,\boldsymbol{w}_k)$，且 $||\boldsymbol{w}_i||_2=1,\boldsymbol{w}_i^T\boldsymbol{w}_j=0,i\neq j$。变换后的新表示可认为是 $\boldsymbol{x}$ 与这 $k$ 个正交基向量内积的组合：$\boldsymbol{x}_i'=(\boldsymbol{w}_1^T\boldsymbol{x}_i,\cdots,\boldsymbol{w}_k^T\boldsymbol{x}_i)$。  
&emsp;&emsp;延续我们在上文的讨论，我们需要使得该变换后的方差 $\lambda$ 最大，则在每一维上的方差都应尽可能大。在第 $j<k$ 维上有：  
$$
\lambda_j=\frac{1}{n-1}\sum\limits_{i=1}^n (\boldsymbol{x}_{ij}'-\overline{\boldsymbol{x}}_j')^2=\frac{1}{n-1}\sum\limits_{i=1}^n  (\boldsymbol{x}_{ij}'- \frac{1}{n-1}\sum\limits_{m=1}^n \boldsymbol{w}_j^T \boldsymbol{x}_m)^2=\frac{1}{n-1}\sum\limits_{i=1}^n (\boldsymbol{x}_{ij}')^2=\frac{1}{n-1}\sum\limits_{i=1}^n(\boldsymbol{w}_j^T\boldsymbol{x}_i\boldsymbol{x}_i^T\boldsymbol{w}_j)=\frac{1}{n-1}\boldsymbol{w}_j^T\boldsymbol{X}\boldsymbol{X}^T\boldsymbol{w}_j=\boldsymbol{w}_j^T \boldsymbol{S}\boldsymbol{w}_j
$$
&emsp;&emsp;同时左乘 $\boldsymbol{w}_j$，有 $\lambda_j \boldsymbol{w}_j=\boldsymbol{S}\boldsymbol{w}_j$，所以$\lambda_i,\boldsymbol{w}_j$ 分别是协方差矩阵 $\boldsymbol{S}$ 的特征值与特征向量。 所以很显然，我们取协方差矩阵 $\boldsymbol{S}$ 的 $k$ 个最大的特征值对应的特征向量，则得到了我们所需要的变换矩阵 $\mathcal{W}$ 。

In [6]:
def my_pca(data, k):
    X_mean = np.mean(data, axis=0)  # (32,)
    data = data - X_mean  # 去均值
    ew, ev = np.linalg.eig(data.T.dot(data))  # 特征值与特征向量, ew:(32,), ev:(32, 32)
    ew_order = np.argsort(ew)[::-1]  #从大到小
    ev_sort = ev[:, ew_order]
    feature = ev_sort[:, :k]
    new_data = data.dot(feature)
    return new_data


my_pca_data = my_pca(matrix, k=5)
my_pca_data.shape

(68040, 5)

In [7]:
my_pca_cov = cov(my_pca_data, bias=False)
my_pca_cov

array([[ 3.10149526e-02,  9.67559554e-17,  8.60255998e-18,
        -3.13295052e-18, -1.02212511e-17],
       [ 9.67559554e-17,  2.76484273e-02,  0.00000000e+00,
        -1.58997239e-17, -1.25579100e-17],
       [ 8.60255998e-18,  0.00000000e+00,  1.93800927e-02,
        -9.41843251e-18,  1.56190638e-17],
       [-3.13295052e-18, -1.58997239e-17, -9.41843251e-18,
         1.57017760e-02,  1.17094026e-17],
       [-1.02212511e-17, -1.25579100e-17,  1.56190638e-17,
         1.17094026e-17,  1.18432912e-02]])

In [33]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
pca.fit(matrix)
standard_pca_data = pca.transform(matrix)
standard_pca_cov = np_cov_func(standard_pca_data.T, bias=False)

In [34]:
compare(my_pca_cov, standard_pca_cov)

mean_dif:1.258848305636457e-10
max_dif:8.731390625407442e-10
min_dif:7.054773432102479e-14


In [35]:
compare(my_pca_data, standard_pca_data)


mean_dif:0.11721605390112377
max_dif:1.2458614313109861
min_dif:1.608102540018308e-12
